In [1]:
import torch
import torch.nn as nn 
import numpy as np 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,TensorDataset
import torch.optim as optim  
from sklearn.model_selection import train_test_split 
random_seed = 42 

##1.Specify each path 

In [2]:
dataset = 'F:\TTCS\model\keypoint_classifier\keypoint.csv'
model_save_path = 'F:\TTCS\model\keypoint_classifier\weights_model.pth'
pth_save_path = 'F:\TTCS\model\keypoint_classifier\converted_model.pth'

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14872\2977927794.py:1: SyntaxWarning: invalid escape sequence '\T'
  dataset = 'F:\TTCS\model\keypoint_classifier\keypoint.csv'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14872\2977927794.py:2: SyntaxWarning: invalid escape sequence '\T'
  model_save_path = 'F:\TTCS\model\keypoint_classifier\weights_model.pth'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14872\2977927794.py:3: SyntaxWarning: invalid escape sequence '\T'
  pth_save_path = 'F:\TTCS\model\keypoint_classifier\converted_model.pth'


In [3]:
n_classes = 5

In [4]:
X_dataset = np.loadtxt(dataset, delimiter = ',', dtype = 'float32', usecols = list(range(1,(21*2)+1)))
y_dataset = np.loadtxt(dataset, delimiter = ',', dtype = 'int32', usecols = (0))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset,y_dataset,test_size = 0.25, random_state = random_seed)

In [6]:
print(len(X_train))
print(len(X_test))

3656
1219


##2.Build the Model

In [7]:
myModel = nn.Sequential(
    nn.Linear(42,20),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.Linear(20,10),
    nn.Dropout(p=0.4),
    nn.ReLU(),
    nn.Linear(10,n_classes),
    nn.Softmax(dim = -1)
)
print(myModel)

Sequential(
  (0): Linear(in_features=42, out_features=20, bias=True)
  (1): Dropout(p=0.2, inplace=False)
  (2): ReLU()
  (3): Linear(in_features=20, out_features=10, bias=True)
  (4): Dropout(p=0.4, inplace=False)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=5, bias=True)
  (7): Softmax(dim=-1)
)


In [8]:
train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train),torch.from_numpy(y_train)),batch_size = 32,shuffle = True)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test),torch.from_numpy(y_test)),batch_size = 32, shuffle = False)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(myModel.parameters(),lr = 0.001)

##3. Train Model

In [11]:
num_epochs = 1000
patience = 20
best_val_loss = float('inf')
counter = 0 

for epoch in range(num_epochs):
    myModel.train()
    for x, y in train_loader:
        output = myModel(x)
        loss = criterion(output,y.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    myModel.eval()
    val_loss = 0.00
    total = 0
    correct = 0.00 
    with torch.no_grad():
        for x,y in test_loader:
            output = myModel(x)
            _, predicted = torch.max(output.data,1)
            loss = criterion(output,y.long())
            val_loss += loss.item()
            correct += (predicted == y).sum().item()
            total += y.size(0)
    val_loss /= len(test_loader)
    accuracy = (100*correct)/total
    print(f'Epoch: {epoch+1}, Val Loss: {val_loss:.4f} Accuracy: {accuracy:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(myModel.state_dict(),'best_model.pth')
        print('ok')
    
            

Epoch: 1, Val Loss: 1.0027 Accuracy: 89.6637
ok
Epoch: 2, Val Loss: 1.0023 Accuracy: 89.8277
ok
Epoch: 3, Val Loss: 1.0017 Accuracy: 89.8277
ok
Epoch: 4, Val Loss: 1.0029 Accuracy: 89.7457
Epoch: 5, Val Loss: 1.0027 Accuracy: 89.7457
Epoch: 6, Val Loss: 1.0015 Accuracy: 89.9098
ok
Epoch: 7, Val Loss: 1.0021 Accuracy: 89.7457
Epoch: 8, Val Loss: 1.0020 Accuracy: 89.9098
Epoch: 9, Val Loss: 1.0025 Accuracy: 89.7457
Epoch: 10, Val Loss: 1.0026 Accuracy: 89.7457
Epoch: 11, Val Loss: 1.0023 Accuracy: 89.8277
Epoch: 12, Val Loss: 1.0025 Accuracy: 89.8277
Epoch: 13, Val Loss: 1.0020 Accuracy: 89.8277
Epoch: 14, Val Loss: 1.0017 Accuracy: 89.9098
Epoch: 15, Val Loss: 1.0025 Accuracy: 89.8277
Epoch: 16, Val Loss: 1.0016 Accuracy: 89.9098
Epoch: 17, Val Loss: 1.0013 Accuracy: 89.9098
ok
Epoch: 18, Val Loss: 1.0017 Accuracy: 89.8277
Epoch: 19, Val Loss: 1.0018 Accuracy: 89.9098
Epoch: 20, Val Loss: 1.0021 Accuracy: 89.7457
Epoch: 21, Val Loss: 1.0020 Accuracy: 89.7457
Epoch: 22, Val Loss: 1.0011 

In [12]:
best_model = torch.load(r'F:\TTCS\model\keypoint_classifier\best_model.pth')
print(best_model)

OrderedDict({'0.weight': tensor([[-9.2397e-02,  2.9384e-02,  3.2992e-02,  4.5138e-01,  1.7233e-01,
          6.6353e-02,  1.4985e-01, -6.1230e-01,  1.2495e-01, -6.4370e-01,
         -1.3329e-01,  3.9762e-01,  8.7608e-02, -9.1438e-03, -3.8680e-02,
         -8.5716e-02,  1.9608e-02, -1.3613e-01, -2.9353e-03,  2.1105e-01,
          4.5788e-02, -3.8452e-01,  2.9907e-02,  2.0438e-02, -1.4651e-01,
          5.8894e-01, -1.6818e-01, -1.6222e-01,  2.8279e-02, -2.9030e-01,
         -2.6488e-01,  1.9776e-01, -1.4271e-01,  1.1305e+00,  9.1884e-03,
         -5.9085e-01,  1.0283e-01, -3.6207e-01, -6.1441e-02,  4.1042e-01,
         -3.9726e-01,  1.1168e+00],
        [-2.8184e-02, -1.3310e-01,  5.4186e-01,  4.6994e-02,  3.6728e-01,
         -2.0945e-01,  1.0815e-01, -9.5666e-02, -1.4874e-01, -1.1126e-01,
          5.6141e-01,  1.7558e-01,  5.4185e-02, -5.0412e-02, -2.4319e-01,
          1.8259e-01, -7.6494e-01,  5.8106e-01,  3.6904e-01,  1.2940e-01,
          1.6873e-01, -2.1426e-01,  3.0097e-01, -4.

C:\Users\Admin\AppData\Local\Temp\ipykernel_14872\285514669.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(r'F:\TTCS\model\keypoint_classifier\b